In [17]:
#! pip3 install --upgrade --user google-cloud-aiplatform

In [94]:
import re
import pandas as pd
from google.cloud import bigquery
import vertexai 
from vertexai.generative_models import GenerationConfig, GenerativeModel, Image, Part
import vertexai.preview.generative_models as generative_models 
import sys
import os
#from dotenv import load_dotenv #

In [95]:
import os 
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\flori\\OneDrive\\Desktop\\LDP\\ldp-data-genai-producto-b871ebffc681.json"

In [96]:
# Define project information
PROJECT_ID = "ldp-data-genai-producto"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


vertexai.init(project=PROJECT_ID, location=LOCATION)

In [97]:
model = GenerativeModel("gemini-1.5-flash-001")

In [98]:
safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
}

In [99]:
generation_config =GenerationConfig(
    temperature=0,
    top_p=0,
    max_output_tokens=8192,
)

In [100]:
# Leer el archivo CSV en una variable como texto
with open('primera-division-salaries-club_salaries.csv', mode='r') as file:
    text = file.read()

# Ahora text es una variable que contiene el contenido del archivo CSV como un string
print(text)


club_name,url,element_id,value
,https://www.capology.comClub,weekly-avg,"Error: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=128.0.6613.120)
Stacktrace:
	GetHandleVerifier [0x00538213+26163]
	(No symbol) [0x004C9CC4]
	(No symbol) [0x003C24C3]
	(No symbol) [0x003BAAD1]
	(No symbol) [0x003AE668]
	(No symbol) [0x003B008B]
	(No symbol) [0x003AE8B7]
	(No symbol) [0x003AE25C]
	(No symbol) [0x003AE188]
	(No symbol) [0x003AC588]
	(No symbol) [0x003ACB1B]
	(No symbol) [0x003C4F04]
	(No symbol) [0x00445107]
	(No symbol) [0x0042ACBC]
	(No symbol) [0x00444661]
	(No symbol) [0x0042AA56]
	(No symbol) [0x003FBE89]
	(No symbol) [0x003FC8CD]
	GetHandleVerifier [0x0080D313+2996019]
	GetHandleVerifier [0x00861B89+3342249]
	GetHandleVerifier [0x005C7AEF+614159]
	GetHandleVerifier [0x005CF17C+644508]
	(No symbol) [0x004D27FD]
	(No symbol) [0x004CF6F8]
	(No symbol) [0x004CF895]
	(No symbol) [0x004C1C16]
	BaseThreadInitThunk [0x76E77BA9+25]
	RtlInitializeExceptionChain [0x7783C1

In [101]:

# Prompt con el contexto y la pregunta
prompt = f"""Quisiera que extraigas del siguiente texto: {text} la informacion relevante que te voy a explicar a continuacion:
Debes colocar en una tabla:
columna1: region, la region de donde son los paises del club en el texto
columna2: pais, el pais de donde son los clubes en el texto
columna3: liga, liga de donde son los clubes que se encuentra en el nombre:{file}
columna4: club: el nombre que esta en el link del texto, 
columna5: el valor numerico weekly-avg sin el simbolo $, € o £ y sin comas
columna6: valor numerico annual-avg sin el simbolo $, € o £ y sin coma
columna7: moneda en que esta el valor numerico si tiene el simbolo $ que contenga la palabra dolares, si contiene el simbolo € que contenga la palabra euros o  si contiene el simbolo £ la palabra libras.
Debes realizar esto con todos los clubes en el texto. El nombre de las columnas debe ser region, pais, liga, club, weeklyavg, annualavg, moneda.
Utiliza informacion unicamente contenida en este texto"""

# Generar la respuesta utilizando el prompt actualizado
responses = model.generate_content(prompt, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
full_text = ""
for response in responses:
    full_text += response.text

print(full_text)

## Salarios de Clubes de Fútbol

| region | pais | liga | club | weeklyavg | annualavg | moneda |
|---|---|---|---|---|---|---|
| Sudamérica | Argentina | Primera División | Argentinos Juniors | 3017 | 156897 | dolares |
| Sudamérica | Argentina | Primera División | Banfield | 3195 | 166129 | dolares |
| Sudamérica | Argentina | Primera División | Barracas | 2897 | 150667 | dolares |
| Sudamérica | Argentina | Primera División | Belgrano | 3821 | 198667 | dolares |
| Sudamérica | Argentina | Primera División | Boca Juniors | 11844 | 615882 | dolares |
| Sudamérica | Argentina | Primera División | Central Cordoba | 1390 | 72286 | dolares |
| Sudamérica | Argentina | Primera División | Defensa y Justicia | 2624 | 136452 | dolares |
| Sudamérica | Argentina | Primera División | Estudiantes | 8033 | 417692 | dolares |
| Sudamérica | Argentina | Primera División | Gimnasia La Plata | 3962 | 206000 | dolares |
| Sudamérica | Argentina | Primera División | Godoy Cruz | 3054 | 158800 | dolares

In [92]:
matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_text)


# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df = pd.DataFrame(matches) 

df = df[~df.eq('---').all(axis=1)]

df.columns = df.iloc[0]  # La primera fila se convierte en nombres de columna
df = df[1:]

# Resetea el índice
df.reset_index(drop=True, inplace=True)

df['weeklyavg'] = df['weeklyavg'].astype('Int64')  # Usar 'Int64' para manejar NaN
df['annualavg'] = df['annualavg'].astype('Int64')

print(df)

0       region       pais              liga                club  weeklyavg  \
0   Sudamérica  Argentina  Primera División  Argentinos Juniors       3017   
1   Sudamérica  Argentina  Primera División            Banfield       3195   
2   Sudamérica  Argentina  Primera División            Barracas       2897   
3   Sudamérica  Argentina  Primera División            Belgrano       3821   
4   Sudamérica  Argentina  Primera División        Boca Juniors      11844   
5   Sudamérica  Argentina  Primera División     Central Cordoba       1390   
6   Sudamérica  Argentina  Primera División  Defensa y Justicia       2624   
7   Sudamérica  Argentina  Primera División         Estudiantes       8033   
8   Sudamérica  Argentina  Primera División   Gimnasia La Plata       3962   
9   Sudamérica  Argentina  Primera División          Godoy Cruz       3054   
10  Sudamérica  Argentina  Primera División             Huracan       4962   
11  Sudamérica  Argentina  Primera División       Independiente 

este seria el codigo equivalente para subir a bigquery pero no se si lo necesitamos aun

In [93]:
# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.primera_division'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  #WRITE_APPEND si quiero sumar
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

Los datos han sido cargados en la tabla ldp-data-wyscout.capology.primera_division.


In [ ]:
### colombia
with open('nombre del csv', mode='r') as file:
    full_text3 = file.read()

# Ahora text es una variable que contiene el contenido del archivo CSV como un string
print(full_text3)

prompt2 = f"""Quisiera que extraigas del siguiente texto: {text} la informacion relevante que te voy a explicar a continuacion:
Debes colocar en una tabla:
columna1: region, la region de donde son los paises del club
columna2: pais, el pais de donde son los clubes en el texto
columna3: liga, liga de donde son los clubes que se encuentra en el nombre:{file}
columna4: club: el nombre que esta en el link del texto, 
columna5: el valor numerico weekly-avg sin el simbolo $, € o £ y sin coma
columna6: valor numerico annual-avg sin el simbolo $, € o £ y sin coma
columna7: moneda en que esta el valor numerico si tiene el simbolo $ que contenga la palabra dolares, si contiene el simbolo € que contenga la palabra euros o  si contiene el simbolo £ la palabra libras.
Debes realizar esto con todos los clubes en el texto. El nombre de las columnas debe ser region, pais, liga, club, weeklyavg, annualavg, moneda.
Utiliza informacion unicamente contenida en este texto"""

# Generar la respuesta utilizando el prompt actualizado
responses2 = model.generate_content(prompt2, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
full_full_text3 = ""
for response2 in responses2:
    full_full_text3 += response2.text

print(full_full_text3)

matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_text)


# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df = pd.DataFrame(matches) 

df = df[~df.eq('---').all(axis=1)]

df.columns = df.iloc[0]  # La primera fila se convierte en nombres de columna
df = df[1:]

# Resetea el índice
df.reset_index(drop=True, inplace=True)

df['weeklyavg'] = df['weeklyavg'].astype('Int64')  # Usar 'Int64' para manejar NaN
df['annualavg'] = df['annualavg'].astype('Int64')

print(df)

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.salarios'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  ### para sumar info WRITE_APPEND
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")

In [ ]:
### colombia
with open('nombre del csv', mode='r') as file:
    full_text3 = file.read()

# Ahora text es una variable que contiene el contenido del archivo CSV como un string
print(full_text3)

prompt2 = f"""Quisiera que extraigas del siguiente texto: {text} la informacion relevante que te voy a explicar a continuacion:
Debes colocar en una tabla:
columna1: region, la region de donde son los paises del club en el texto
columna2: pais, el pais de donde son los clubes en el texto
columna3: liga, liga de donde son los clubes que se encuentra en el nombre:{file}
columna4: club: el nombre que esta en el link del texto, 
columna5: el valor numerico weekly-avg sin el simbolo $, € o £ y sin coma
columna6: valor numerico annual-avg sin el simbolo $, € o £ y sin coma
columna7: moneda en que esta el valor numerico si tiene el simbolo $ que contenga la palabra dolares, si contiene el simbolo € que contenga la palabra euros o  si contiene el simbolo £ la palabra libras.
Debes realizar esto con todos los clubes en el texto. El nombre de las columnas debe ser region, pais, liga, club, weeklyavg, annualavg, moneda.
Utiliza informacion unicamente contenida en este texto"""

# Generar la respuesta utilizando el prompt actualizado
responses2 = model.generate_content(prompt2, generation_config=generation_config, stream=True)

# Imprimir la respuesta generada
full_full_text3 = ""
for response2 in responses2:
    full_full_text3 += response2.text

print(full_full_text3)

matches = re.findall(r'\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|\s*(.*?)\s*\|', full_text) #separa en 7 col


# Convierte los datos en un DataFrame de pandas y asigna las columnas correctamente
df = pd.DataFrame(matches) 

df = df[~df.eq('---').all(axis=1)]

df.columns = df.iloc[0]  # La primera fila se convierte en nombres de columna
df = df[1:]

# Resetea el índice
df.reset_index(drop=True, inplace=True)

df['weeklyavg'] = df['weeklyavg'].astype('Int64')  # Usar 'Int64' para manejar NaN
df['annualavg'] = df['annualavg'].astype('Int64')

print(df)

# Especifica tu ID de proyecto
project_id = 'ldp-data-wyscout'

# Crear un cliente de BigQuery con el proyecto especificado
client = bigquery.Client(project=project_id)

# Especificar la tabla de destino en el formato dataset.tabla
table_id = 'ldp-data-wyscout.capology.salarios'

# Configurar la carga de datos
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,  ### para sumar info WRITE_APPEND
)

# Guardar el DataFrame en BigQuery
job = client.load_table_from_dataframe(df2, table_id, job_config=job_config)
job.result()  # Esperar a que el trabajo termine

print(f"Los datos han sido cargados en la tabla {table_id}.")